Testa själv kod

In [77]:
# For Python 2 / 3 compatability
from __future__ import print_function

In [78]:
training_data = [
    [70, 19.8,'Klassrum'],
    [54.4, 14.4, 'Kylrum'],
    [45.4, 12.4, 'Kylrum'],
    [89.4, 19.5, 'Klassrum'],
    [57.4, 18.1, 'Lärarrum'],
     [22.4, 8.6, 'Kylrum'],
    [24.4, 11.24, 'Kylrum'],
    [84.4, 24.4, 'Klassrum'],
    [95.4, 22.4, 'Klassrum'],
    [81.4, 20.1, 'Lärarrum'],
    [70, 19.7, 'Lärarrum'],
]

In [79]:
# Column labels.
# These are used only to print the tree.
header = ["Humidity", "Heat", "label"]

In [80]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [81]:
unique_vals(training_data, 2)

{'Klassrum', 'Kylrum', 'Lärarrum'}

In [82]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [83]:
class_counts(training_data)

{'Klassrum': 4, 'Kylrum': 4, 'Lärarrum': 3}

In [84]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [85]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        #Nya metoden, tittar så det är ett nummer man skickar in returnerar true om det är större än frågans värde och false om det är mindre.
    def moreless(self, val):
        if is_numeric(val):
            return val >= self.value
        else:
            return val <= self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [86]:
# Skapar en fråga där vi säger är värmen  större eller mindre än vårt värde [0][1] alltså rad 0 värde 1 alltså 14.4
q = Question(1, training_data[0][1])
q

Is Heat >= 19.8?

In [87]:

example = 11#sätter 11 och kollar om det är mer eller mindre än valt värde
q.moreless(example)

False

In [88]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

def partitionMoreLess(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
#         print(row,question.column)
        if question.moreless(row[question.column]):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [89]:
true_rows, false_rows = partitionMoreLess(training_data, Question(1, 20)) #kolumn 1 (Heat),över 20 grader
true_rows

[[84.4, 24.4, 'Klassrum'], [95.4, 22.4, 'Klassrum'], [81.4, 20.1, 'Lärarrum']]

In [90]:
false_rows

[[70, 19.8, 'Klassrum'],
 [54.4, 14.4, 'Kylrum'],
 [45.4, 12.4, 'Kylrum'],
 [89.4, 19.5, 'Klassrum'],
 [57.4, 18.1, 'Lärarrum'],
 [22.4, 8.6, 'Kylrum'],
 [24.4, 11.24, 'Kylrum'],
 [70, 19.7, 'Lärarrum']]

In [91]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [92]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [93]:
current_uncertainty = gini(training_data)
current_uncertainty

0.6611570247933884

In [94]:
# How much information do we gain by partioning on 20 heat?
true_rows, false_rows = partitionMoreLess(training_data, Question(1, 20))
info_gain(true_rows, false_rows, current_uncertainty)

0.08539944903581265

In [95]:
# How much information do we gain by partioning on labeln kylrum?
true_rows, false_rows = partitionMoreLess(training_data, Question(2, "Kylrum"))
info_gain(true_rows, false_rows, current_uncertainty)

0.2975206611570248

In [96]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partitionMoreLess(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [97]:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)
best_question


Is Heat >= 18.1?

In [98]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [99]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [100]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [101]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [102]:
my_tree = build_tree(training_data)

In [103]:
print_tree(my_tree)

Is Heat >= 18.1?
--> True:
  Is Humidity >= 84.4?
  --> True:
    Predict {'Klassrum': 3}
  --> False:
    Is Heat >= 19.8?
    --> True:
      Is Heat >= 20.1?
      --> True:
        Predict {'Lärarrum': 1}
      --> False:
        Predict {'Klassrum': 1}
    --> False:
      Predict {'Lärarrum': 2}
--> False:
  Predict {'Kylrum': 4}


In [104]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [105]:
#######
# Demo:
# The tree predicts the 1st row of our
# training data is an apple with confidence 1.
classify(training_data[4], my_tree)
#######

{'Lärarrum': 2}

In [106]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [107]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(classify(training_data[0], my_tree))
#######

{'Klassrum': '100%'}

In [108]:
#######
# Demo:
# On the second example, the confidence is lower
print_leaf(classify(training_data[8], my_tree))
#######

{'Klassrum': '100%'}

In [109]:
# Evaluate
testing_data = [
    [30, 15.6, 'Kylrum'],
    [50, 16, 'Klassrum'],
    [89, 21, 'Klassrum'],
    [81.7, 19, 'Lärarrum'],
    [87, 19.8, 'Lärarrum'],
]

In [110]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Kylrum. Predicted: {'Kylrum': '100%'}
Actual: Klassrum. Predicted: {'Kylrum': '100%'}
Actual: Klassrum. Predicted: {'Klassrum': '100%'}
Actual: Lärarrum. Predicted: {'Lärarrum': '100%'}
Actual: Lärarrum. Predicted: {'Klassrum': '100%'}


In [111]:
import random
import pandas as pd
def Createvalues():
    values=[]
    mall=[{"label":"Kylrum","minheat":6.4,"maxheat": 14.34,"minhum":20,"maxhum":56.6},{"label":"Klassrum","minheat":18.4,"maxheat": 24.34,"minhum":70,"maxhum":96.6},{"label":"Lärarrum","minheat":17.4,"maxheat": 23.65,"minhum":65,"maxhum":93}]
    for typavrum in mall:
        antalgånger= random.randint(1,101)
        for i in range(0,antalgånger):
            nytemp= random.uniform(typavrum["minheat"],typavrum["maxheat"])
            nyhum= random.uniform(typavrum["minhum"],typavrum["maxhum"])
#             value= {"hum":nyhum,"heat":nytemp,"label":typavrum["label"]}
            value= [nyhum,nytemp,typavrum["label"]]
            values.append(value)

    print(values[0])
    print(values[2])
    print(len(values))
    return values

    #List of dictionaries
newdata= Createvalues()
#Dataframe
newvalues=pd.DataFrame(newdata)


[37.19219693598917, 10.253751179359956, 'Kylrum']
[25.250447911285292, 7.561411773530645, 'Kylrum']
178


In [112]:
# newvalues.info()
# newvalues.describe()
# newvalues.label.value_counts()
newvalues[2].value_counts()


Lärarrum    97
Klassrum    42
Kylrum      39
Name: 2, dtype: int64

In [113]:

print(newvalues.head())
newvalues = newvalues.sample(frac=1).reset_index(drop=True)
print(newvalues.head())


           0          1       2
0  37.192197  10.253751  Kylrum
1  33.439154  10.662700  Kylrum
2  25.250448   7.561412  Kylrum
3  20.743508   8.197611  Kylrum
4  45.095719  13.596210  Kylrum
           0          1         2
0  90.031014  21.658571  Lärarrum
1  78.024164  22.592034  Klassrum
2  69.583943  21.969126  Lärarrum
3  75.805788  23.235558  Klassrum
4  74.538427  22.046420  Klassrum


In [114]:
def Splitdata(inputdata,n_trees,ownvalue): 
#     print(len(inputdata),n_trees, ownvalue)
    subsets=[]
    splitone=inputdata.sample(frac=0.5,random_state=200)
    splittwo=inputdata.drop(splitone.index)
    if n_trees!=1:
        feedback= Splitdata(splitone,n_trees-1,ownvalue+"a")
        feedback2 =Splitdata(splittwo,n_trees-1,ownvalue+"b")
        for i in range(0,len(feedback)):
#             print(i,"Len",len(feedback[i]), n_trees,ownvalue)
#             print(i,"Len",len(feedback2[i]), n_trees,ownvalue)
            subsets.append(feedback[i])
            subsets.append(feedback2[i])  
        
    else:
        return [splitone,splittwo]
    return subsets

Subsets =Splitdata(newvalues,4,"a")

print("Antal Subsets:",len(Subsets))
print("Antal värden i varje subset:",len(Subsets[0]))
print(Subsets[0])

Antal Subsets: 16
Antal värden i varje subset: 11
             0          1         2
10   73.391155  18.299832  Lärarrum
59   71.973221  19.460391  Lärarrum
102  21.279077   9.921223    Kylrum
34   70.869904  19.944357  Lärarrum
21   86.421350  18.393916  Lärarrum
2    69.583943  21.969126  Lärarrum
166  75.141328  19.934480  Lärarrum
125  88.683096  22.661902  Lärarrum
75   77.939215  22.225468  Lärarrum
109  87.503339  19.112737  Lärarrum
145  81.992477  17.849203  Lärarrum


In [115]:
My_trees=[]
i=1
for Myset in Subsets:
    Mylist= Myset.values.tolist()
    print("Träd",i)
    i+=1
    tree=build_tree(Mylist)
    My_trees.append(tree)
    print_tree(tree)
    


Träd 1
Is Heat >= 17.849202851410894?
--> True:
  Predict {'Lärarrum': 10}
--> False:
  Predict {'Kylrum': 1}
Träd 2
Is Heat >= 19.06675655995003?
--> True:
  Is Heat >= 19.622332592320237?
  --> True:
    Is Heat >= 23.650348128961042?
    --> True:
      Predict {'Klassrum': 1}
    --> False:
      Predict {'Lärarrum': 6}
  --> False:
    Predict {'Klassrum': 1}
--> False:
  Predict {'Kylrum': 3}
Träd 3
Is Heat >= 17.841313111600563?
--> True:
  Is Humidity >= 93.41294468300745?
  --> True:
    Predict {'Klassrum': 1}
  --> False:
    Predict {'Lärarrum': 8}
--> False:
  Predict {'Kylrum': 2}
Träd 4
Is Heat >= 19.373909999396353?
--> True:
  Is Heat >= 21.792047291668656?
  --> True:
    Predict {'Klassrum': 2}
  --> False:
    Is Humidity >= 92.13409539325218?
    --> True:
      Predict {'Klassrum': 1}
    --> False:
      Is Humidity >= 85.92836608576748?
      --> True:
        Predict {'Lärarrum': 3}
      --> False:
        Is Heat >= 20.992065293049397?
        --> True:
     

In [116]:
def Predictioncounter(row,trees,labels):
    length= len(labels)
    predictions=[0 for i in range(length)]
    for tree in trees:
        prediction= classify(row, tree)
        for i in range(length):
             if(prediction.get(labels[i])is not None):
                predictions[i]+=1
#         print ("Actual: %s. Predicted: %s" %
#                (row[-1], print_leaf(classify(row, tree))))
    return predictions

from tqdm import tqdm

def Randomforest(trees,data):
    for row in tqdm(data):
        predictions=[0,0,0]
        labels=["Kylrum","Lärarrum","Klassrum"]
        predictions= Predictioncounter(row,trees,labels)
        high=max(predictions)
        print("Actual:",row[2],labels[predictions.index(high)],":", max(predictions),"/",len(trees))
                            
Randomforest(My_trees,testing_data)

100%|██████████| 5/5 [00:00<00:00, 621.89it/s]

Actual: Kylrum Kylrum : 15 / 16
Actual: Klassrum Kylrum : 15 / 16
Actual: Klassrum Lärarrum : 11 / 16
Actual: Lärarrum Lärarrum : 9 / 16
Actual: Lärarrum Lärarrum : 13 / 16
